# TASK 1
riconoscimento review positiva o negativa

In [1]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras import Sequential

import tensorflow as tf

import libraries.preprocessing_utils as prep_utils
from libraries.dataset import Dataset

# import libraries.utils as utils
import constants as const
import os

Data retrieving + preprocessing

In [2]:
review_data = Dataset('review', 'sentiment')
review_data.split(['text'], 'sentiment', n_samples=500_000)

Reading ./data/balanced_review_sentiment_train.csv...
File loaded in 0.2 minutes
Reading ./data/balanced_review_sentiment_val.csv...
File loaded in 0.0 minutes
Reading ./data/balanced_review_sentiment_test.csv...
File loaded in 0.0 minutes


In [3]:
# preprocess text -> list[str]
# fit tokenizer and tokenize
tokenizer = prep_utils.get_tokenizer(review_data.train_data[0]['text'])

train_tokens = prep_utils.get_set_tokens(
    review_data.train_data[0]['text'], tokenizer, set='train', task='task1')

test_tokens = prep_utils.get_set_tokens(
    review_data.test_data[0]['text'], tokenizer, set='test', task='task1')

val_tokens = prep_utils.get_set_tokens(
    review_data.val_data[0]['text'], tokenizer, set='val', task='task1')


In [19]:
# create embedding matrix

# get embedded matrix based containing vectors from a pretrained dict
# vectors are related only to words found in train sentences
e_matrix = prep_utils.get_embedding_matrix(const.word_embedding_filepath, 'task1',
                                            tokenizer, len(tokenizer.index_word)+1)

Reading pretrained word embedding from ./data/embedding/glove.twitter.27B.100d.txt...
	0 words loaded
	50000 words loaded
	100000 words loaded
	150000 words loaded
	200000 words loaded
	250000 words loaded
	300000 words loaded
	350000 words loaded
	400000 words loaded
	450000 words loaded
	500000 words loaded
	550000 words loaded
	600000 words loaded
	650000 words loaded
	700000 words loaded
	750000 words loaded
	800000 words loaded
	850000 words loaded
	900000 words loaded
	950000 words loaded
	1000000 words loaded
	1050000 words loaded
	1100000 words loaded
	1150000 words loaded
Creating embedding matrix...
	61.0% words vector not found (169916 over 278570)
...embedding matrix created (matrix pickled at ./data/embedding/task1_embedding_matrix.npy)


## RNN

In [21]:
#  TODO
embedding_size = 32

word_vector_dim = 100
vocab_size = len(tokenizer.word_index) +1

max_length = len(max(train_tokens, key=len))

embedding_layer = Embedding(vocab_size, word_vector_dim,
                        embeddings_initializer=Constant(e_matrix), trainable=False)

model = Sequential()
model.add(embedding_layer)  # the embedding layer
model.add(LSTM(15, dropout=0.5))
model.add(Dense(1,activation='sigmoid'))


model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_tokens, review_data.train_data[1], epochs=5,validation_data=(val_tokens, review_data.val_data[1]))



278569
278570
